## Connect DynamoDB using spark connector from EMR Studio Notebook using Spark Scala

#### Topics covered in this example

* Configuring the EMR-DynamoDB Connector
* Connecting to AWS DynamoDB using EMR-DynamoDB Connector to read data into Spark DF and get the count

## Table of Contents:

1. [Prerequisites](#Prerequisites)
2. [Introduction](#Introduction)
3. [Load the configuration in memory](#Load-the-configuration-in-memory)
4. [Read data using Scala](#Read-data-using-Scala)

## Prerequisites

 1. EMR Version - emr-6.4.0
 2. The notebook is using Amazon EMR-DynamoDB Connector(emr-ddb-hadoop.jar) which is used to access data in Amazon DynamoDB using Apache Hadoop, Apache Hive, and Apache Spark in Amazon EMR. (https://docs.aws.amazon.com/emr/latest/ReleaseGuide/EMRforDynamoDB.html)
 3. The Amazon EMR-DynamoDB Connector is locally available within EMR at location below: 
 
 `/usr/share/aws/emr/ddb/lib/emr-ddb-hadoop.jar`
 4. To locally use the spark.jars in the configuration using the `file://` protocol, Livy does not allow local files to be uploaded to the user session by default. So we need to update the livy configuration to allow the local directory and restart livy server to have these take effects.
     
     a. Open the Livy configuration file
     
     `vim /etc/livy/conf/livy.conf`
     
     b. Add the ddb jars:
     
     `livy.file.local-dir-whitelist   /usr/share/aws/emr/ddb/lib/`
     
     c. Restart Levi
     
     `systemctl restart livy-server.service`



## Introduction

This notebook shows how to connect to DynamoDB using DynamoDB Spark connector(emr-ddb-hadoop) from Amazon EMR Studio Notebook using Scala

## Load the configuration in memory

In [ ]:
%%configure -f
{
    "conf": {
        "spark.jars":"file:///usr/share/aws/emr/ddb/lib/emr-ddb-hadoop.jar" 
    }
}

## Read data using Pyspark

In [ ]:
import org.apache.hadoop.io.Text;
import org.apache.hadoop.dynamodb.DynamoDBItemWritable
import org.apache.hadoop.dynamodb.read.DynamoDBInputFormat
import org.apache.hadoop.dynamodb.write.DynamoDBOutputFormat
import org.apache.hadoop.mapred.JobConf
import org.apache.hadoop.io.LongWritable

var jobConf = new JobConf(sc.hadoopConfiguration)
jobConf.set("dynamodb.servicename", "dynamodb")
jobConf.set("dynamodb.input.tableName", "<table_name>")   // Pointing to DynamoDB table
jobConf.set("dynamodb.endpoint", "<dynamodb_endpoint>")
jobConf.set("dynamodb.regionid", "<aws_region>")
jobConf.set("dynamodb.throughput.read", "1")
jobConf.set("dynamodb.throughput.read.percent", "1")
jobConf.set("dynamodb.version", "2011-12-05")
 
jobConf.set("mapred.output.format.class", "org.apache.hadoop.dynamodb.write.DynamoDBOutputFormat")
jobConf.set("mapred.input.format.class", "org.apache.hadoop.dynamodb.read.DynamoDBInputFormat")
 
var orders = sc.hadoopRDD(jobConf, classOf[DynamoDBInputFormat], classOf[Text], classOf[DynamoDBItemWritable])
 
// Doing a count of items on Orders table
orders.count()